In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt

# デバイスの設定
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 1. CNNの定義
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(64 * 7 * 7, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = x.view(-1, 64 * 7 * 7)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# 2. データの準備
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=True)

# 3. モデルと最適化関数の初期化
model = CNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 4. モデルの学習（5エポック）
epochs = 5
model.train()
for epoch in range(epochs):
    total_loss = 0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    avg_loss = total_loss / len(train_loader)
    print(f'Epoch [{epoch + 1}/{epochs}], Loss: {avg_loss:.4f}')

# 5. 予測関数
def get_prediction(model, image):
    model.eval()
    with torch.no_grad():
        output = model(image)
    pred = torch.argmax(output, dim=1)
    return pred.item()

# 6. FGSMでの敵対的サンプル生成
def generate_adversarial_example(model, image, label, epsilon=0.3):
    # 2025-02-14 16:57 epsilon=0.3に変更
    image.requires_grad = True
    output = model(image)
    loss = criterion(output, label)
    model.zero_grad()
    loss.backward()

    # ノイズ生成
    perturbation = epsilon * image.grad.sign()
    adversarial_image = image + perturbation
    adversarial_image = torch.clamp(adversarial_image, 0, 1)

    adversarial_pred = get_prediction(model, adversarial_image)
    return adversarial_image, perturbation, adversarial_pred

# 7. 10回試行して攻撃成功回数をカウント
success_count = 0
attempts = 10

for i, (image, label) in enumerate(test_loader):
    if i >= attempts:
        break  # 10回試行で終了

    image, label = image.to(device), label.to(device)
    
    # 敵対的サンプル生成
    adversarial_image, perturbation, adversarial_pred = generate_adversarial_example(model, image, label)

    # 予測結果取得
    original_pred = get_prediction(model, image)

    if original_pred != adversarial_pred:
        success_count += 1
        print(f"試行 {i+1}: 成功！元の予測: {original_pred} → 敵対的サンプルの予測: {adversarial_pred}")
    else:
        print(f"試行 {i+1}: 失敗 元の予測: {original_pred} → 敵対的サンプルの予測: {adversarial_pred}")

# 8. 攻撃成功回数を表示
print(f"\nFGSM攻撃の成功回数: {success_count} / {attempts}")


Epoch [1/5], Loss: 0.1398
Epoch [2/5], Loss: 0.0460
Epoch [3/5], Loss: 0.0308
Epoch [4/5], Loss: 0.0210
Epoch [5/5], Loss: 0.0164
試行 1: 失敗 元の予測: 8 → 敵対的サンプルの予測: 8
試行 2: 失敗 元の予測: 8 → 敵対的サンプルの予測: 8
試行 3: 失敗 元の予測: 1 → 敵対的サンプルの予測: 1
試行 4: 失敗 元の予測: 9 → 敵対的サンプルの予測: 9
試行 5: 失敗 元の予測: 8 → 敵対的サンプルの予測: 8
試行 6: 失敗 元の予測: 1 → 敵対的サンプルの予測: 1
試行 7: 失敗 元の予測: 1 → 敵対的サンプルの予測: 1
試行 8: 失敗 元の予測: 3 → 敵対的サンプルの予測: 3
試行 9: 失敗 元の予測: 7 → 敵対的サンプルの予測: 7
試行 10: 失敗 元の予測: 3 → 敵対的サンプルの予測: 3

FGSM攻撃の成功回数: 0 / 10
